In [1]:
!pip install qiskit qiskit-aer --quiet


from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import matplotlib.pyplot as plt
import pandas as pd

def deutsch_oracle(qc, function_type):
    """Oracle Uf for given function type."""
    if function_type == 'constant_0':
        pass
    elif function_type == 'constant_1':
        qc.x(1)
    elif function_type == 'balanced_0':
        qc.cx(0, 1)
    elif function_type == 'balanced_1':
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)
    elif function_type == 'xor_1':
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00


In [2]:
def deutsch_algorithm(function_type, visualize=True):
    qc = QuantumCircuit(2, 1)
    qc.x(1)
    qc.h([0, 1])
    if visualize:
        state1 = Statevector(qc)
        print("→ After Hadamard gates:")
        display(plot_bloch_multivector(state1))

    deutsch_oracle(qc, function_type)
    if visualize:
        state2 = Statevector(qc)
        print("→ After Oracle:")
        display(plot_bloch_multivector(state2))

    qc.h(0)
    qc.measure(0, 0)

    sim = AerSimulator()
    counts = sim.run(transpile(qc, sim)).result().get_counts()

    result = 'CONSTANT' if max(counts, key=counts.get) == '0' else 'BALANCED'
    print(f"\nFunction Type: {function_type}\nMeasurement: {counts} → {result}")
    plot_histogram(counts)
    plt.show()
    return result, counts

functions = ['constant_0', 'constant_1', 'balanced_0', 'balanced_1', 'xor_1']
results = []
for f in functions:
    res, cnt = deutsch_algorithm(f, visualize=False)
    results.append((f, list(cnt.keys())[0], res))

df = pd.DataFrame(results, columns=["Function Type", "Measurement", "Result"])
print("\n--- Result Comparison ---")
print(df.to_string(index=False))





Function Type: constant_0
Measurement: {'0': 1024} → CONSTANT

Function Type: constant_1
Measurement: {'0': 1024} → CONSTANT

Function Type: balanced_0
Measurement: {'1': 1024} → BALANCED

Function Type: balanced_1
Measurement: {'1': 1024} → BALANCED

Function Type: xor_1
Measurement: {'1': 1024} → BALANCED

--- Result Comparison ---
Function Type Measurement   Result
   constant_0           0 CONSTANT
   constant_1           0 CONSTANT
   balanced_0           1 BALANCED
   balanced_1           1 BALANCED
        xor_1           1 BALANCED


In [3]:
def deutsch_jozsa_2qubit(is_balanced=True):
    qc = QuantumCircuit(3, 2)
    qc.x(2)
    qc.h([0, 1, 2])

    if is_balanced:
        qc.cx(0, 2)
        qc.cx(1, 2)
    else:
        pass

    qc.h([0, 1])
    qc.measure([0, 1], [0, 1])

    sim = AerSimulator()
    counts = sim.run(transpile(qc, sim)).result().get_counts()
    plot_histogram(counts)
    plt.show()
    print("Balanced" if any('1' in bit for bit in counts) else "Constant")

print("\n--- Deutsch–Jozsa Extension (2-qubit) ---")
deutsch_jozsa_2qubit(is_balanced=True)



--- Deutsch–Jozsa Extension (2-qubit) ---
Balanced


In [4]:
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.05, 1), ['h', 'x'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.02, 2), ['cx'])

qc = QuantumCircuit(2, 1)
qc.x(1)
qc.h([0, 1])
deutsch_oracle(qc, 'balanced_0')
qc.h(0)
qc.measure(0, 0)

sim_noisy = AerSimulator(noise_model=noise_model)
result_noisy = sim_noisy.run(transpile(qc, sim_noisy)).result()
counts_noisy = result_noisy.get_counts()

print("\n--- Noise Impact ---")
plot_histogram(counts_noisy)
plt.show()
print("Noisy measurement:", counts_noisy)



--- Noise Impact ---
Noisy measurement: {'0': 88, '1': 936}


In [5]:
def classical_check(fx):
    return "CONSTANT" if fx(0) == fx(1) else "BALANCED"

fx_dict = {
    'constant_0': lambda x: 0,
    'constant_1': lambda x: 1,
    'balanced_0': lambda x: x,
    'balanced_1': lambda x: 1 - x,
    'xor_1': lambda x: x ^ 1
}

comparison = []
for f in functions:
    classical_res = classical_check(fx_dict[f])
    quantum_res, _ = deutsch_algorithm(f, visualize=False)
    comparison.append((f, classical_res, quantum_res))

df_compare = pd.DataFrame(comparison, columns=["Function", "Classical Result", "Quantum Result"])
print("\n--- Classical vs Quantum Comparison ---")
print(df_compare.to_string(index=False))


Function Type: constant_0
Measurement: {'0': 1024} → CONSTANT

Function Type: constant_1
Measurement: {'0': 1024} → CONSTANT

Function Type: balanced_0
Measurement: {'1': 1024} → BALANCED

Function Type: balanced_1
Measurement: {'1': 1024} → BALANCED

Function Type: xor_1
Measurement: {'1': 1024} → BALANCED

--- Classical vs Quantum Comparison ---
  Function Classical Result Quantum Result
constant_0         CONSTANT       CONSTANT
constant_1         CONSTANT       CONSTANT
balanced_0         BALANCED       BALANCED
balanced_1         BALANCED       BALANCED
     xor_1         BALANCED       BALANCED
